In [10]:
import datetime as dt
import utils as ut
import pandas as pd
import icharts as ic
import holoviews as hv
import base as b
import hvplot.pandas  # noqa
from functools import cache
from logger_settings import logger
from constants import *
from bokeh.plotting import figure, show, output_notebook
output_notebook()


test_date = dt.datetime.strptime("2024-01-01", "%Y-%m-%d")
SYMBOL = "NIFTY 50"
IC_SYMBOL = "NIFTY"
INTERVAL = ut.INTERVAL_MIN1
EXCHANGE = ut.EXCHANGE_NSE
MONITORING_STRIKE_PRICE = 120
TRADING_STRIKE_PRICE = 40

TEST_START = dt.datetime.strptime("2024-01-01", "%Y-%m-%d")
TEST_END = dt.datetime.strptime("2024-01-10", "%Y-%m-%d")

train_dates, test_dates = ut.get_date_range(start_date=TEST_START, end_date=TEST_END, symbol=SYMBOL, interval=INTERVAL, exchange=EXCHANGE)
train_dates["expiry"] = pd.NA
train_dates["expiry"] = train_dates.apply(lambda row: ut.find_nclosest_expiry(SYMBOL, row.name, 1), axis=1)

train_dates['call_trading'] = train_dates.apply(lambda r: ut.get_strike_price_by_price(symbol=SYMBOL, expiry=r.expiry, timestamp=r.name.replace(hour=9, minute=15), option_type=OPTION_TYPE_CALL, price=TRADING_STRIKE_PRICE, exchange=EXCHANGE), axis=1)
train_dates['put_trading'] = train_dates.apply(lambda r: ut.get_strike_price_by_price(symbol=SYMBOL, expiry=r.expiry, timestamp=r.name.replace(hour=9, minute=15), option_type=OPTION_TYPE_PUT, price=TRADING_STRIKE_PRICE, exchange=EXCHANGE), axis=1)
train_dates['call_monitoring'] = train_dates.apply(lambda r: ut.get_strike_price_by_price(symbol=SYMBOL, expiry=r.expiry, timestamp=r.name.replace(hour=9, minute=15), option_type=OPTION_TYPE_CALL, price=MONITORING_STRIKE_PRICE, exchange=EXCHANGE), axis=1)
train_dates['put_monitoring'] = train_dates.apply(lambda r: ut.get_strike_price_by_price(symbol=SYMBOL, expiry=r.expiry, timestamp=r.name.replace(hour=9, minute=15), option_type=OPTION_TYPE_PUT, price=MONITORING_STRIKE_PRICE, exchange=EXCHANGE), axis=1)

from typing import Dict

def update_strike(ps, call_df, put_df, till_idx, call_strike, put_strike):
    ps.call_ticks = None
    ps.put_ticks = None
    ps.meta = {'call_strike': call_strike, 'put_strike': put_strike}

    for i in range(till_idx + 1):
        cdict = call_df.iloc[i].to_dict()
        cdict['timestamp'] = call_df.iloc[i].name
        pdict = put_df.iloc[i].to_dict()
        pdict['timestamp'] = put_df.iloc[i].name
        ps.next_no_order(cdict, pdict)


settings = {
    # Order Settings
    "quantity": 25,
}


# Get nifty candles for minute
# For first minute, get premium strike price for call and puts which is price at ~30 
# Draw them here 
# Try to match the pattern 

Loading BokehJS ...

In [2]:
class PeakStrategy(b.Strategy):
    def __init__(self, instrument: "Instrument", settings: Dict, meta: Dict ={}):
        super().__init__(instrument, settings)
        self.call_ticks: pd.DataFrame | None = None
        self.put_ticks: pd.DataFrame | None = None
        self.meta = meta
        self.om = b.OrderManager()

    def calculate_data(self):
        '''
        '''
        self.put_ticks['open_bps'] = self.put_ticks.open.diff() * 10000 / self.put_ticks.open.shift(1)
        self.put_ticks['high_bps'] = (self.put_ticks.high - self.put_ticks.open) * 10000 / self.put_ticks.open
        self.put_ticks['low_bps'] = (self.put_ticks.open - self.put_ticks.low) * 10000 / self.put_ticks.open
        self.put_ticks['close_bps'] = (self.put_ticks.close - self.put_ticks.open) * 10000 / self.put_ticks.open
        self.put_ticks['wprice'] = (self.put_ticks.high + self.put_ticks.low + 2 * self.put_ticks.close) / 4
        self.put_ticks['size'] = self.put_ticks.close - self.put_ticks.open
        self.put_ticks['is_green'] = self.put_ticks.close >= self.put_ticks.open
        self.put_ticks['is_small'] = self.put_ticks.close_bps < 1

    def is_volume_match(self) -> bool:
        '''
        1. Must have high volume on the last green candle
        2. Must have low volume on the last red candle
        '''
        return True

    def is_price_match(self):
        return True

    def entry_conditions(self) -> bool:
        '''
        Make sure price is not stagnant and has a certain direction since I'll be betting on opposite direction
        Make sure that volume suddenly drops significantly to last volume
        '''
        last_tick = self.put_ticks.iloc[-1]
        last_n_ticks = self.put_ticks[-self.MIN_GREEN_CANDLE_LENGTH-1:-1]
        is_price_match = self.is_price_match()
        is_volume_match = self.is_volume_match()
        if not is_volume_match:
            return False
        if not is_price_match:
            return False
        print(f"matched at: {self.put_ticks.iloc[-1].timestamp}")
        return True

    def exit_conditions(self) -> bool:
        last_tick = self.call_ticks.iloc[-1]
        closep = last_tick.close
        if last_tick.timestamp.hour == 15 and last_tick.timestamp.minute == 28:
            return True, closep
        wprice = last_tick.wprice
        openp = last_tick.open
        for order in self.om.orders:
            if order.created_at == last_tick.timestamp:
                continue
            if (order.limit_price - openp) >= order.meta['sl']:
                return True, order.limit_price - order.meta['sl']
            if (wprice - order.limit_price) >= order.meta['tp']:
                return True, order.limit_price + order.meta['tp']
            if (openp - order.limit_price) >= order.meta['tp']:
                return True, order.limit_price + order.meta['tp']
            if (order.limit_price - closep) >= order.meta['sl']:
                return True, order.limit_price - order.meta['sl']
        return False, None

    def _process_tick(self, call_tick: Dict, put_tick: Dict) -> None:
        call_tick_df = pd.DataFrame(
            call_tick, index=[len(self.call_ticks) if self.call_ticks is not None else 0]
        )
        self.call_ticks = pd.concat([self.call_ticks, call_tick_df], ignore_index=True)

        put_tick_df = pd.DataFrame(
            put_tick, index=[len(self.put_ticks) if self.put_ticks is not None else 0]
        )
        self.put_ticks = pd.concat([self.put_ticks, put_tick_df], ignore_index=True)

    def next_no_order(self, call_tick: Dict, put_tick: Dict):
        self._process_tick(call_tick, put_tick)
        self.calculate_data()

    def next(self, call_tick: Dict, put_tick: Dict):
        self.next_no_order(call_tick, put_tick)
        if not self.om.has_intrade_orders() and self.entry_conditions():
            price_diff = self.call_ticks.iloc[-5:].open.max() - self.call_ticks.iloc[-5:].close.min()
            qv_max = self.call_ticks.iloc[-5:].qv_ratio.max()
            target_pc = qv_max * (self.TARGET_PC_MAX - self.TARGET_PC_MIN ) / (self.MAX_QV_THRESHOLD - self.MIN_QV_THRESHOLD)
            target = self.call_ticks.iloc[-1].close * target_pc
            stoploss = self.call_ticks.iloc[-1].close - self.call_ticks.iloc[-1].low + self.call_ticks.iloc[-5:].low.std()
            # stoploss = min(stoploss, target)
            target = max(stoploss, target)
            order = b.Order(type=b.Order.TYPE_BUY, limit_price=self.call_ticks.iloc[-1].close, created_at=self.call_ticks.iloc[-1].timestamp,
                            quantity=self.quantity, exchange_order_id=None, 
                            meta={'tp': target, 'sl': stoploss, 'call_strike': self.meta['call_strike'], 'put_strike': self.meta['put_strike']})
            self.om.place_order(order)
        is_exit, eprice = self.exit_conditions()
        if self.om.has_intrade_orders() and is_exit:
            self.om.square_off_all_orders(index=self.call_ticks.iloc[-1].timestamp, last_price=eprice)

In [3]:
instrument = b.Instrument(name="NIFTY 22650 CALL 7 Mar 2024")
ps = PeakStrategy(instrument=instrument, settings=settings)

for index, row in train_dates.iterrows():
    trading_df = ic.get_opt_pre_df(symbol=IC_SYMBOL, expiry=row.expiry, cur_dt=index.date(), strike_price=row.call_trading, option_type=OPTION_TYPE_CALL)
    monitoring_df = ic.get_opt_pre_df(symbol=IC_SYMBOL, expiry=row.expiry, cur_dt=index.date(), strike_price=row.put_trading, option_type=OPTION_TYPE_PUT)
    ps.meta={'trading_strike': row['call_trading'], "trading_type": OPTION_TYPE_CALL, 'mon_strike': row['put_monitoring']}
    for i in range(put_df.shape[0]):
        if not ps.om.has_intrade_orders() and (abs(trading_df.iloc[i].close - TRADING_STRIKE_PRICE) < 10) and not (trading_df.iloc[i].name.hour == 15 and trading_df.iloc[i].name.minute >= 29):
            old_price = trading_df.iloc[i].close
            trade_strike = ut.get_strike_price_by_price(symbol=SYMBOL, expiry=row.expiry, timestamp=trading_df.iloc[i].name, option_type=OPTION_TYPE_CALL, price=TRADING_STRIKE_PRICE, exchange=EXCHANGE)
            if trade_strike != row["call_strike"]:
                print(f"changing call strike new: {call_strike}, old: {row['call_strike']}")
                row["call_strike"] = train_dates.loc[index, "call_strike"] = call_strike
                call_df = ic.get_opt_pre_df(symbol=IC_SYMBOL, expiry=row.expiry, cur_dt=index.date(), strike_price=row.call_strike, option_type=OPTION_TYPE_CALL)
                update_strike(ps=ps, call_df=call_df, put_df=put_df, till_idx=i-1, call_strike=row['call_strike'], put_strike=row['put_strike'])
        if not ps.om.has_intrade_orders() and (put_df.iloc[i].close < 18 or put_df.iloc[i].close > 40) and not (call_df.iloc[i].name.hour == 15 and call_df.iloc[i].name.minute >= 29):
            old_price = put_df.iloc[i].close
            put_strike = ut.get_strike_price_by_price(symbol=SYMBOL, expiry=row.expiry, timestamp=put_df.iloc[i].name, option_type=OPTION_TYPE_PUT, price=30, exchange=EXCHANGE)
            if put_strike != row["put_strike"]:
                print(f"changing put strike new: {put_strike}, old: {row['put_strike']}")
                row["put_strike"] = train_dates.loc[index, "put_strike"]  = put_strike
                put_df = ic.get_opt_pre_df(symbol=IC_SYMBOL, expiry=row.expiry, cur_dt=index.date(), strike_price=row.put_strike, option_type=OPTION_TYPE_PUT)
                update_strike(ps=ps, call_df=call_df, put_df=put_df, till_idx=i-1, call_strike=row['call_strike'], put_strike=row['put_strike'])
        cdict = call_df.iloc[i].to_dict()
        cdict['timestamp'] = call_df.iloc[i].name
        pdict = put_df.iloc[i].to_dict()
        pdict['timestamp'] = put_df.iloc[i].name
        # cdict, pdict = pdict, cdict
        ps.next(cdict, pdict)
    break

matched at: 2024-01-01 09:15:00


AttributeError: 'DataFrame' object has no attribute 'qv_ratio'

In [6]:
pcs = [order.pnl_pc for order in ps.om.closed_orders]
print(f"PnL Per Order: {sum(pcs) / len(pcs)}, success rate: {len([x for x in pcs if x > 0]) * 100 / len(pcs)}%")
pcs

PnL Per Order: -7.875738581537783, success rate: 0.0%


[-7.335635578917155, -8.415841584158413]

In [5]:
i = 0
print(ps.om.closed_orders[i].meta)
print(ps.om.closed_orders[i])
print(f"Buy: {ps.om.closed_orders[i].limit_price}, Sell: {ps.om.closed_orders[i].square_off_price} TP: {ps.om.closed_orders[i].limit_price + ps.om.closed_orders[i].meta['tp']}, SL: {ps.om.closed_orders[i].limit_price - ps.om.closed_orders[i].meta['sl']}")

IndexError: list index out of range

In [17]:
call_df = ic.get_opt_pre_df(symbol=IC_SYMBOL, expiry=train_dates.iloc[0].expiry, cur_dt=train_dates.iloc[0].name.date(), strike_price=train_dates.iloc[0].call_strike, option_type=OPTION_TYPE_CALL)
call_df.open - row.call_strike + data.open

2024-01-01 09:15:00   -230.05
2024-01-01 09:16:00   -257.95
2024-01-01 09:17:00   -272.10
2024-01-01 09:18:00   -260.00
2024-01-01 09:19:00   -266.15
                        ...  
2024-01-01 15:26:00   -295.40
2024-01-01 15:27:00   -293.50
2024-01-01 15:28:00   -292.10
2024-01-01 15:29:00       NaN
2024-01-01 15:30:00       NaN
Name: open, Length: 376, dtype: float64

In [19]:
row = train_dates.iloc[0]
data = ut.get_data(symbol=SYMBOL, date=row.name.date(), interval=INTERVAL_MIN1, exchange=EXCHANGE)

call_df = ic.get_opt_pre_df(symbol=IC_SYMBOL, expiry=train_dates.iloc[0].expiry, cur_dt=train_dates.iloc[0].name.date(), strike_price=train_dates.iloc[0].call_strike, option_type=OPTION_TYPE_CALL)
put_df = ic.get_opt_pre_df(symbol=IC_SYMBOL, expiry=train_dates.iloc[0].expiry, cur_dt=train_dates.iloc[0].name.date(), strike_price=train_dates.iloc[0].put_strike, option_type=OPTION_TYPE_PUT)

call_df.open = row.call_strike - data.open - call_df.open
call_df.close = row.call_strike - data.close - call_df.close
call_df.high = row.call_strike - data.high - call_df.high
call_df.low = row.call_strike - data.low - call_df.low

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.models import CDSView, ColumnDataSource, IndexFilter, BooleanFilter

charts, _ = ut.create_candlestick_plot(call_df, title='Call', plot=False)
layout = gridplot(charts)
show(layout)

In [11]:
# call_df = ic.get_opt_pre_df(symbol=IC_SYMBOL, expiry=train_dates.iloc[0].expiry, cur_dt=train_dates.iloc[0].name.date(), strike_price=ps.om.closed_orders[i].meta['call_strike'], option_type=OPTION_TYPE_CALL)
# put_df = ic.get_opt_pre_df(symbol=IC_SYMBOL, expiry=train_dates.iloc[0].expiry, cur_dt=train_dates.iloc[0].name.date(), strike_price=ps.om.closed_orders[i].meta['put_strike'], option_type=OPTION_TYPE_PUT)

call_df = ic.get_opt_pre_df(symbol=IC_SYMBOL, expiry=train_dates.iloc[0].expiry, cur_dt=train_dates.iloc[0].name.date(), strike_price=train_dates.iloc[0].call_strike, option_type=OPTION_TYPE_CALL)
put_df = ic.get_opt_pre_df(symbol=IC_SYMBOL, expiry=train_dates.iloc[0].expiry, cur_dt=train_dates.iloc[0].name.date(), strike_price=train_dates.iloc[0].put_strike, option_type=OPTION_TYPE_PUT)

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.models import CDSView, ColumnDataSource, IndexFilter, BooleanFilter

charts, _ = ut.create_candlestick_plot(call_df, title='Call', plot=False)
layout = gridplot(charts)
show(layout)
charts, cds = ut.create_candlestick_plot(put_df, title='Put', plot=False)
p = charts[0][0]
# view = CDSView(filter=BooleanFilter(ps.put_ticks.red_qualify.values))
# p.scatter(x='date_time', y='close', view=view, source=cds, color='yellow', size=8, marker='circle')
# charts[0][0] = p
layout = gridplot(charts)
show(layout)

In [20]:
cds.data.keys()

dict_keys(['date_time', 'open', 'high', 'low', 'close', 'volume', 'unknown1', 'unknown2', 'unknown3', 'unknown4'])

In [25]:
pd.set_option('display.max_rows', 200)
# ps.put_ticks.loc[ps.put_ticks.red_qualify][["timestamp", "qv", "qv_ratio", "qv_qualify", "prev_ratio", "prev1_ratio", "prev_qv_qualify", "prev1_qv_qualify", "red_qualify", "cdiff"]].head(200)
ps.put_ticks[["timestamp", "qv", "qv_ratio", "qv_qualify", "prev_ratio", "prev1_ratio", "prev_qv_qualify", "prev1_qv_qualify", "red_qualify", "cdiff"]].head(200)

,timestamp,qv,qv_ratio,qv_qualify,prev_ratio,prev1_ratio,prev_qv_qualify,prev1_qv_qualify,red_qualify,cdiff
0,2024-01-01 09:15:00,NaN,NaN,False,NaN,NaN,NaN,NaN,False,NaN
1,2024-01-01 09:16:00,NaN,NaN,False,1.142393,NaN,False,NaN,False,0.3875
2,2024-01-01 09:17:00,NaN,NaN,False,1.114773,1.273509,False,False,False,-0.8500
3,2024-01-01 09:18:00,NaN,NaN,False,0.831007,0.926384,False,False,False,-0.6375
4,2024-01-01 09:19:00,NaN,NaN,False,1.502077,1.248237,False,False,False,0.1375
5,2024-01-01 09:20:00,NaN,NaN,False,0.764939,1.148998,False,False,False,0.0375
6,2024-01-01 09:21:00,NaN,NaN,False,0.500786,0.383071,False,False,False,0.0000
7,2024-01-01 09:22:00,NaN,NaN,False,0.737288,0.369224,False,False,False,-0.2250
8,2024-01-01 09:23:00,NaN,NaN,False,0.800341,0.590082,False,False,False,-0.0250
9,2024-01-01 09:24:00,NaN,NaN,False,0.948936,0.759472,False,False,False,-0.5375


In [20]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.datasets import electrocardiogram
from scipy.signal import find_peaks
row = train_dates.iloc[0]
x = ic.get_opt_pre_df(symbol=IC_SYMBOL, expiry=row.expiry, cur_dt=row.name.date(), strike_price=row.call_strike, option_type=OPTION_TYPE_CALL)
x.loc[:, "peak"] = False
for i in range(x.shape[0]):
    ht = x.iloc[-i].close * .005
    peaks, _ = find_peaks(-x.iloc[:i+1].close, threshold=ht)
    filtered = peaks[peaks >= (i - 2)]
    if len(filtered) > 0:
        x.loc[x.iloc[filtered].index, "peak"] = True

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.models import CDSView, ColumnDataSource, IndexFilter, BooleanFilter

charts, cds = ut.create_candlestick_plot(x, title='Call', plot=False)
p = charts[0][0]
view = CDSView(filter=BooleanFilter(x.peak.values))
p.scatter(x='date_time', y='close', view=view, source=cds, color='yellow', size=8, marker='circle')
charts[0][0] = p
layout = gridplot(charts)
show(layout)

In [30]:
x.loc[x.peak]

,open,high,low,close,volume,unknown1,unknown2,unknown3,unknown4,peak
date_time,,,,,,,,,,
